# Purpose

Upload

The purpose of this notebook is to add a definition of a start up to the affiliation data. There are multiple ways of doing this so the approach taken is as follows:

1. Filter affiliations so that only potential start-ups go through the proceeding steps
2. Use CH API to retrieve registered company numbers for the matches that were made through CH
3. Use company numbers in the batch search functionality of FAME
4. Retrieve information from 2010 on revenue
5. Exclude the top x'th percentile from candidate start-ups

__Resources:__

* [FAME](https://library.bath.ac.uk/management/company-information) - access through university library
* [Companies House API](https://developer-specs.company-information.service.gov.uk/companies-house-public-data-api/reference)

# 0) Import dependencies and load data

In [49]:
import pandas as pd
import numpy as np
import requests
import os
import glob
import constants
from rapidfuzz import process
from tqdm import tqdm
import time

In [2]:
affils = pd.read_csv(os.getcwd() + r'\final_data\scopus_affils_clean.csv')

# 1) Filter affiliations

* `affil_type = company`
* Do not include string matches - if I include Scopus and GRID matches, I will need to do fuzzy matching (on all names - current and previous) before taking the company number to ensure they are the same company
* Have an established date after 1990

In [3]:
# Corresponds to 11,518 affiliations
candidates = affils[(affils['affil_type'] == 'company') &
                    (affils['type_source'] != 'string') &
                    (affils['est_inc_date'] >= 1990)].copy()

# 2) Retrieve registered company numbers

In [4]:
# Add a column to store company number
candidates['co_num'] = None
candidates['best_score'] = None
candidates['best_match'] = None
candidates['CH_city'] = None
candidates['CH_post_code'] = None

In [5]:
def handle_string(string):
    '''Function removes ltd, limited, plc from incoming string to
    improve matching.'''
    
    string = string.lower()
    if ' ltd' in string:
        return string.replace(' ltd', '')
    elif ' limited' in string:
        return string.replace(' limited', '')
    elif ' plc' in string:
        return string.replace(' plc', '')
    else:
        return string
    

# Test - faster than alternatives tried
% timeit handle_string('Strategem PLC')

980 ns ± 51.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [66]:
def get_ch_data(query, key, testing=False):
    
    res = {'co_num': None,
           'prev_names': [''],
           'address': {'city': None,
                       'post_code': None}}
    
    # URLs
    url_search = 'https://api.company-information.service.gov.uk/search/companies'
    url_profile = 'https://api.company-information.service.gov.uk/company/?'
    
    # Retrieve company number and address
    connected = False
    while not connected:
        try:
            search_resp = requests.get(url_search, params = {'q': query}, auth = (key, ''))
            if search_resp.status_code == 200:
                connected = True
        except:
            time.sleep(10)
            pass
    search_resp = search_resp.json()
    if search_resp['total_results'] == 0:
        return res
    res['co_num'] = search_resp['items'][0]['company_number']
    address = search_resp['items'][0]['address']
    
    # Address can be None if not provided
    if address is not None:
        if 'locality' in address:
            res['address']['city'] = address['locality']
        if 'postal_code' in address:
            res['address']['post_code'] = address['postal_code']
    
    # Find all names - previous and current
    connected = False
    while not connected:
        try:
            profile_resp = requests.get(url_profile.replace('?', res['co_num']), auth = (key, ''))
            if profile_resp.status_code == 200:
                connected = True
        except:
            time.sleep(10)
            pass
    names = [profile_resp.json()['company_name']]
    if 'previous_company_names' in profile_resp.json():
        names += [prev['name'] for prev in profile_resp.json()['previous_company_names']]
    
    # Remove ltd, limited, plc to improve matching performance
    res['prev_names'] = [handle_string(name) for name in names]

    return res


# Test
#% timeit get_ch_data('starlight express limited', constants.API_KEY_CH)

In [67]:
for row in tqdm(candidates.itertuples(), total=candidates.shape[0]):
    
    if pd.notnull(getattr(row, 'co_num')):
        continue
    
    # Get company number and all previous names
    orig_name = handle_string(getattr(row, 'affil_name'))
    ret = get_ch_data(orig_name, constants.API_KEY_CH)
    
    # If match wasn't through CH, check fuzzy matching - CHECK ANYWAY
    result = process.extract(orig_name, ret['prev_names'], limit=1)[0]
    best_name, best_score = result[0], result[1]
    
    # If fuzzy match good OR original match was CH >>
    # add company number to candidates
    candidates.at[row[0], 'co_num'] = ret['co_num']
    candidates.at[row[0], 'best_score'] = best_score
    candidates.at[row[0], 'best_match'] = best_name
    candidates.at[row[0], 'CH_city'] = ret['address']['city']
    candidates.at[row[0], 'CH_post_code'] = ret['address']['post_code']

100%|██████████████████████████████████████████████████████████████████████████| 11518/11518 [2:01:48<00:00,  1.58it/s]


In [70]:
candidates.CH_city.isnull().value_counts()

False    11202
True       316
Name: CH_city, dtype: int64

In [72]:
candidates.to_csv('scopus_affils_startup.csv')

In [2]:
candidates = pd.read_csv('scopus_affils_startup.csv')

In [4]:
candidates.drop(columns=['Unnamed: 0.1'], inplace=True)

# 3) Create csv files for FAME batch search

In [15]:
print('{} out of {} will be go to FAME.'.format(candidates[candidates['best_score'] > 85].shape[0], 
                                               candidates.shape[0]))

11386 out of 11518 will be go to FAME.


In [23]:
# Create new directory
os.mkdir('FAME')
os.chdir(os.getcwd() + '\\FAME')

In [48]:
candidates[candidates['best_score'] > 85]['co_num'].to_csv('big_boy.csv', index=False)

# 4) Load in revenue data

Note: There were duplicates in the original data set so there are fewer entries in the resulting FAME search.

In [143]:
fname = glob.glob('*.xlsx')
rev = pd.read_excel(fname[0], sheet_name=1)

In [144]:
drop = ['Unnamed: 0', 'Quoted', 'Branch', 'OwnData', 'Woco', 'R/O Full Postcode']
rev.drop(columns=drop, inplace=True)

In [145]:
# 2009 rev - 3667 nulls
# Last available year - 6169 nulls
# 2009 #employees - 3995 nulls
rev.rename(columns={'Operating revenue (Turnover)\nth GBP 2009': '2009_revenue',
                    'Turnover\nth GBP Last avail. yr': 'last_revenue',
                    'Number of employees\n2009': '2009_employees'},
          inplace=True)

In [146]:
# Change n.a to np.nan
for row in rev.itertuples():
    idx = getattr(row, 'Index')
    rev_2009 = getattr(row, '_6')
    
    # Change string to np.nan
    if rev_2009 == 'n.a.':
        rev.at[idx, '2009_revenue'] = np.nan

In [150]:
rev = rev.astype({'2009_revenue': float})

# 5) Exclude x'th percentile

97th percentile corresponds to £42,192.20 in revenue. `idx = 265`.

In [158]:
# Sort by 20009 revenue
rev.sort_values(by=['2009_revenue'], ascending=False, inplace=True)

In [202]:
# Find a percentile to cut off - happy with 97th percentile
percentile = 97
idx = round(rev.shape[0] * ((100 - percentile) / 100))

# Add column for start-up
rev['start_up'] = None
rev.iloc[:idx, -1] = False
rev.iloc[idx:, -1] = True

In [205]:
# Save new csv
rev.to_csv('start-ups.csv', index=False)

# 6) Merge files together

In [215]:
# Drop irrelevant columns on revenue df
rev = rev[['Registered number', 'start_up']]

# Merge on company number
rev.rename(columns={'Registered number': 'co_num'}, inplace=True)
merged = pd.merge(candidates, rev, how='left', on='co_num')

In [234]:
# For companies that didn't match on FAME, set to start-up
merged['start_up'].fillna(value=True, inplace=True)

In [240]:
# Merge with affils df
affils = pd.read_csv('..\\final_data\\scopus_affils_clean.csv')

In [244]:
columns = ['af_id', 'start_up']
final = pd.merge(affils, merged[columns], how='left', on='af_id')

In [247]:
# Set start up to False for the rest of affils
final['start_up'].fillna(value=False, inplace=True)

# Save final csv

In [249]:
final.to_csv('..\\final_data\\affils_w_startup.csv')